In [6]:
import numpy as np
import open3d as o3d
import yaml
import MinkowskiEngine as ME

/home/arash/ht/minkowski-building-segmentation/MinkowskiEngine/__init__.py:36: UserWarning: The environment variable `OMP_NUM_THREADS` not set. MinkowskiEngine will automatically set `OMP_NUM_THREADS=16`. If you want to set `OMP_NUM_THREADS` manually, please export it on the command line before running a python script. e.g. `export OMP_NUM_THREADS=12; python your_program.py`. It is recommended to set it below 24.
  warnings.warn(


In [5]:


# TODO: 
# quantize the data into a sparse tensor: MinkowskiEngine.utils.sparse_quantize
# Tensor: ME.SparseTensor

def load_label_color_map(filepath):
    try:
        with open(filepath) as yamlfile:
            label_color = yaml.load(yamlfile, Loader=yaml.FullLoader)
            print(f"Loaded {len(label_color)} labels from {filepath}")
    except FileNotFoundError:
        raise ValueError(f"{filepath} not found.")
    return label_color

def mp(mapper_dict, entry):
    """ map class labels to RGB colors
    Args:
        mapper_dict: dictionary loaded from yaml file
        entry: int representing the label
        default color is black
    """
    colors = mapper_dict.get(entry, (0, 0, 0))
    return tuple(colors)

def out_pcd(coords, features):
    """
    Point cloud from numpy arrays of shape (n, 3)
    Args:
        coords: Numpy array of shape (n, 3) with 3D coordinates of points
        features: Numpy array of shape (n, 3) with colors
    """

    pcd = o3d.geometry.PointCloud()
    # checks if it contains normals, if so save them
    #print('Does PCD file have normals? ', pcd.has_normals())
    # false
    # normalize_normals()
    # returns: open3d.geometry.PointCloud
    pcd.points = o3d.utility.Vector3dVector(coords)
    #print('Does PCD file have points? ', pcd.has_points())
    pcd.colors = o3d.utility.Vector3dVector(features)
    #print('Does PCD file have colors? ', pcd.has_colors())
    return pcd

def preprocess(pcd_file, voxel_size, visualize = False, save = False):
    # Load the point cloud
    pcd = o3d.io.read_point_cloud(pcd_file, remove_nan_points=True, remove_infinite_points=True)
    asc_file = "{}{}" .format(pcd_file[:-4], ".asc")
    pcd_arr = np.loadtxt(asc_file, delimiter=";")
    labels = pcd_arr[:,6].astype(int)

    # Normalize the point cloud
    origin = np.array([0.0, 0.0, 0.0])
    #print(origin.shape)
    ##(3,)
    # move point cloud to origin
    pcd.translate(origin)
    
    scale_factor = 1.0 / np.max(np.abs(pcd.points))
    #print(scale_factor)
    ## 0.08629493049708449
    pcd.scale(scale_factor, origin)

    # Voxel downsample point cloud = 15% of original size
    # Normals and colors are averaged if they exist
    down_pcd = pcd.voxel_down_sample(voxel_size)

    # compute normals
    radius_normal = voxel_size * 2
    # estimate_normals(self, search_param=KDTreeSearchParamKNN with knn = 30, fast_normal_computation=True)
    down_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    #o3d.visualization.draw_geometries([down_pcd],
    #                              point_show_normal=True)

    #down_pcd.normalize_normals()
    # returns: o3d.geometry.PointCloud
        
    #down_pcd.remove_radius_outlier(nb_points=16, radius=0.05)
    #down_pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
    # returns: Tuple[open3d.geometry.PointCloud, List[int]]
    
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)
    # number of points
    num_points_down_pcd = len(down_pcd.points)

    # assign labels based on the nearest label
    label_list = []
    for i in range(num_points_down_pcd):
        # search_knn_vector_3d(self, query, knn)
        # more dynamic way: search_hybrid_vector_3d(self, query, radius, max_nn)
        [k, idx, _] = pcd_tree.search_knn_vector_3d(down_pcd.points[i], 1)
        index = np.asarray(idx)[0]
        label_list.append(labels[index])
        
    # Find the most frequent label among the neighbors and add it to the label list
    #from statistics import mode
    #label_list = []
    #for i in range(num_points_down_pcd):
    #    k = 5
    #    [k, idx, _] = pcd_tree.search_knn_vector_3d(down_pcd.points[i], k)
    #    neighbor_labels = [labels[index] for index in idx]
    #    most_frequent_label = mode(neighbor_labels)
    #    label_list.append(most_frequent_label)    

    print("Label list length: ", len(label_list))
    #label_list_arr = np.array(label_list)
    #print(label_list_arr.shape)

    # coords, values, labels arrays
    coords = np.asarray(down_pcd.points)
    #print("Shape of coords array: ", coords.shape)
    values = np.asarray(down_pcd.colors) # use real colours
    #print("Shape of values array: ", values.shape)
    labels = np.asarray(label_list, dtype=int)
    #print("Shape of labels array: ", labels.shape)
    #for e in np.nditer(labels):
    #    print(e)
    normals = np.asarray(down_pcd.normals)
    print('Does PCD file have normals? ', down_pcd.has_normals())
    print ("Shape of normals array: ", normals.shape)

    # label color point cloud
    label_dict = load_label_color_map("label-color-map.yaml")
    #print(label_dict)
    # needs two arguments: function and iterable
    map = np.vectorize(mp)  
    #print('map', map)
    ##map <numpy.vectorize object at 0x7f4605b19f40>  
    label_colors = np.asarray(map(label_dict, labels))
    #print('before transpose: ', label_colors.shape)
    ## (3, 37294)
    label_colors = np.transpose(label_colors)
    #print('after transpose: ', label_colors.shape)
    ## after transpose (37294, 3)
    
    # normalize colours
    label_colors = label_colors / 255
    #print(label_colors)
    #(rows, cols) = label_colors.shape
    #for i in range(rows):
    #    for j in range(cols):
    #        print(label_colors[i][j], end=" ")
    #    print()

    out_label_pcd = out_pcd(coords, label_colors)
    # save pointcloud for checks
    if save: 
        label_out_file = "{}{}".format(pcd_file[:-4], "proc_labeled.ply", print_progress = True)
        o3d.io.write_point_cloud(label_out_file, out_label_pcd)
    if visualize:
        o3d.visualization.draw_geometries([out_label_pcd])

    # real color point cloud
    out_color_pcd = out_pcd(coords, values)
    if save: 
        color_out_file = "{}{}".format(pcd_file[:-4], "proc_textured.ply", print_progress = True)
        o3d.io.write_point_cloud(color_out_file, out_label_pcd)
    if visualize:
        o3d.visualization.draw_geometries([out_color_pcd])


    # create sparse tensor
    sparse_tensor = ME.SparseTensor(coords, values, labels)

    # Augment the data
    sparse_tensor = sparse_tensor.augment(augmentations={'rotation':[0, 90, 180, 270]})

if __name__=="__main__":
    preprocess("construction_small.pcd", voxel_size=0.01, visualize=True, save=False)


num_points_pcd:  249572
num_points_down_pcd:  37294
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
1
1
1
1
1
1
1
1
1
1
1
101
1
101
1
1
101
101
101
4
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
101
101
101
101
101
101
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
101
101
101
4
4
4
4
4
4
4
4
4
4
101
101
101
101
4
4
4
101
10

12
12
12
12
1
12
12
4
12
12
12
12
12
4
12
4
12
12
12
12
12
12
1
12
12
12
12
12
12
101
12
12
12
12
12
12
12
1
12
12
101
12
12
12
101
12
12
12
12
12
12
101
12
12
12
12
12
12
12
12
101
12
12
12
12
12
12
12
12
12
12
12
12
12
4
12
12
101
12
12
12
12
12
12
12
12
4
12
4
12
12
12
1
12
12
12
12
12
12
12
12
12
12
101
1
12
12
12
101
12
12
101
1
12
101
12
101
12
4
12
101
12
4
12
4
1
12
4
12
4
12
101
12
12
4
4
12
12
4
12
4
4
1
12
12
12
4
4
4
4
12
4
12
12
12
12
4
4
12
4
12
12
101
4
12
12
4
12
12
4
12
4
4
4
12
4
12
1
4
4
12
1
4
12
12
12
12
4
4
4
4
4
12
1
12
12
12
12
4
4
12
4
4
4
12
12
12
4
1
4
12
12
4
12
12
4
12
12
12
12
12
101
101
12
4
12
101
12
12
12
12
12
101
12
4
12
4
12
1
4
12
12
12
4
4
12
12
12
4
4
4
4
4
4
12
4
12
12
12
4
4
4
12
12
12
101
4
1
12
12
12
4
12
12
12
12
4
4
4
12
4
12
4
101
12
12
12
12
12
12
101
12
101
12
12
4
101
12
101
12
12
4
4
4
101
4
4
4
12
101
12
12
12
4
4
12
12
12
12
12
4
12
12
101
12
101
4
4
12
12
12
4
101
12
12
101
12
12
12
12
12
4
12
101
12
12
12
12
12
12
12
12
12
12
12
12


100
1
101
4
8
1
0
1
4
101
12
101
12
101
101
4
101
1
1
4
4
1
1
1
1
101
4
1
4
101
0
100
4
1
12
4
4
100
0
1
4
4
101
1
1
1
1
4
12
4
1
1
101
100
12
4
100
4
1
0
4
4
100
12
4
4
101
1
101
101
1
100
4
1
4
4
0
100
100
4
4
1
1
1
101
4
1
1
1
101
101
100
8
4
4
1
4
12
1
1
101
100
0
12
8
4
12
101
101
101
4
101
4
12
4
12
4
1
4
4
4
1
101
12
1
1
1
12
8
4
100
101
4
4
12
100
4
12
1
1
100
4
12
1
100
4
1
1
101
100
4
1
100
4
4
101
12
1
1
1
0
8
12
101
4
1
12
4
1
100
4
4
1
1
4
101
1
4
1
12
1
4
101
1
1
1
12
4
101
12
1
4
4
1
8
1
12
101
1
1
101
100
101
1
1
1
12
0
12
4
101
101
100
4
101
101
4
101
1
101
1
1
1
100
1
101
1
101
4
101
4
12
101
4
12
4
4
8
1
12
101
1
4
100
101
101
101
12
12
4
100
4
12
100
101
4
12
4
11
101
4
12
101
12
4
101
12
101
1
4
1
1
1
101
12
1
101
12
4
4
100
1
1
101
1
1
1
1
1
0
1
1
101
4
101
1
4
12
4
1
1
100
12
4
0
4
1
4
11
12
1
4
12
101
101
4
101
1
4
101
12
4
1
12
101
4
4
101
12
12
4
101
8
1
100
4
4
4
1
1
4
101
100
1
1
1
101
101
4
4
1
1
1
12
100
1
1
4
1
0
1
4
1
1
0
1
4
1
1
1
4
4
101
1
4
1
1
12
101

101
100
1
8
4
100
100
12
1
12
101
0
1
1
101
4
1
1
1
1
4
12
4
1
1
1
101
1
4
101
4
8
1
1
100
1
101
101
4
1
4
12
100
101
4
101
101
100
1
101
101
1
1
101
101
100
4
4
101
101
4
100
101
1
1
12
1
101
12
1
101
1
4
101
1
12
101
100
101
100
11
1
4
101
4
1
1
101
4
4
4
101
4
11
12
1
1
4
12
101
1
101
101
4
12
101
100
0
4
1
1
101
101
12
4
1
1
12
101
101
1
1
1
1
101
12
101
100
101
12
101
1
1
1
1
1
1
12
12
0
12
12
1
8
1
1
100
12
101
12
1
101
12
4
101
12
1
12
4
101
1
12
101
1
1
100
12
12
0
101
0
1
1
1
1
4
4
12
0
12
1
101
1
12
1
1
101
4
4
4
101
12
101
4
12
4
1
12
4
12
4
100
8
12
4
1
4
101
12
4
1
101
12
4
101
12
12
4
4
4
4
4
4
4
12
1
4
4
1
101
101
1
1
12
101
101
4
12
100
101
101
12
4
101
12
1
12
4
12
1
4
12
1
12
1
12
1
1
4
12
4
101
4
12
12
101
12
101
4
1
0
12
101
1
1
12
12
101
100
11
4
12
1
1
4
1
12
100
101
1
4
12
100
4
1
1
1
12
1
1
12
101
0
0
101
101
0
12
12
4
101
1
1
12
4
4
101
12
4
1
1
4
100
12
4
4
0
4
0
4
4
12
101
4
12
12
4
1
100
101
1
1
12
12
100
4
1
12
1
12
1
1
12
100
12
12
12
100
101
12
8
12
100
0

Loaded 34 labels from label-color-map.yaml
